# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [27]:
weather = pd.read_csv ("weather_info.csv")
weather = weather.drop("Unnamed: 0", axis = 1)
weather

,Cities,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country
0,kapaa,78.80,22.08,-159.32,74,17.22,90,US
1,busselton,45.00,-33.65,115.33,70,5.10,0,AU
2,faya,66.20,18.39,42.45,55,2.62,57,SA
3,avarua,80.60,-21.21,-159.78,65,10.29,40,CK
4,lima,64.40,-12.04,-77.03,77,10.29,75,PE
...,...,...,...,...,...,...,...,...
538,nantucket,52.00,41.28,-70.10,53,12.75,1,US
539,iquitos,73.40,-3.75,-73.25,94,5.82,75,PE
540,manokwari,82.06,-0.87,134.08,72,1.92,83,ID
541,mbanza-ngungu,68.99,-5.25,14.87,99,4.61,0,CD


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
# Configure gmaps
gmaps.configure(api_key=g_key)

from ipywidgets.embed import embed_minimal_html
locations = weather[['Lat', 'Lng']]
weights=weather['Humidity']


# map types ['ROADMAP', 'HYBRID', 'TERRAIN', 'SATELLITE']
fig = gmaps.figure(map_type='TERRAIN', center=(0,0), zoom_level =3) 

heat_layer = gmaps.heatmap_layer(locations, weights=weights,max_intensity=0,point_radius = 30)

# Add layer
fig.add_layer(heat_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

##### Create new DataFrame fitting weather criteria
* Make sure max temperature is not higher than 75 or lower than 65
* Ensure cloudiness is 0
* Wind is less than 10 mph

In [35]:
vacation_weather = weather.loc[(weather["Max Temp"]>60)&(weather["Max Temp"]<=70)]
vacation_weather = vacation_weather.loc[vacation_weather["Wind Speed"]<=10]
vacation_weather = vacation_weather.loc[vacation_weather["Cloudiness"]==0]
vacation_weather = vacation_weather.fillna("NM")
vacation_weather

,Cities,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country
31,luderitz,69.80,-26.65,15.16,26,3.36,0,NM
96,venancio aires,66.99,-29.61,-52.19,80,3.44,0,BR
195,opuwo,61.03,-18.06,13.84,40,6.91,0,NM
204,richards bay,64.62,-28.78,32.04,90,5.19,0,ZA
219,ambilobe,66.49,-13.20,49.05,82,3.51,0,MG
273,tigzirt,67.33,36.89,4.12,53,4.81,0,DZ
276,ahipara,60.13,-35.17,173.17,65,8.01,0,NZ
310,antsohihy,69.84,-14.88,47.99,69,8.21,0,MG
349,yafran,68.27,32.06,12.53,34,9.55,0,LY
351,medea,60.80,36.26,2.75,72,2.24,0,DZ


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotel_df = vacation_weather.copy()
hotel_df["Hotel Name"] = ''
hotel_df = hotel_df.rename(columns = {"Cities":"City"})
hotel_df

,City,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country,Hotel Name
31,luderitz,69.80,-26.65,15.16,26,3.36,0,NM,
96,venancio aires,66.99,-29.61,-52.19,80,3.44,0,BR,
195,opuwo,61.03,-18.06,13.84,40,6.91,0,NM,
204,richards bay,64.62,-28.78,32.04,90,5.19,0,ZA,
219,ambilobe,66.49,-13.20,49.05,82,3.51,0,MG,
273,tigzirt,67.33,36.89,4.12,53,4.81,0,DZ,
276,ahipara,60.13,-35.17,173.17,65,8.01,0,NZ,
310,antsohihy,69.84,-14.88,47.99,69,8.21,0,MG,
349,yafran,68.27,32.06,12.53,34,9.55,0,LY,
351,medea,60.80,36.26,2.75,72,2.24,0,DZ,


In [42]:

target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index,row in hotel_df.iterrows():
    city = row["City"]
    lat = row["Lat"]
    long = row["Lng"]
    params = {
        "location": f'{lat},{long}',
        "radius":5000,
        "keyword":"hotel",
        "key": g_key
    }


    response = requests.get(target_url, params).json()

    results = response['results']
    
    try:
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

        
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "No Hotel Found within 5000 m"
        


In [43]:
hotel_df
hotel_df = hotel_df.rename(columns = {"Cities": "City"})
hotel_df

,City,Max Temp,Lat,Lng,Humidity,Wind Speed,Cloudiness,Country,Hotel Name
31,luderitz,69.80,-26.65,15.16,26,3.36,0,NM,LÜDERITZ NEST HOTEL
96,venancio aires,66.99,-29.61,-52.19,80,3.44,0,BR,Hotel Guest
195,opuwo,61.03,-18.06,13.84,40,6.91,0,NM,Opuwo Country Lodge
204,richards bay,64.62,-28.78,32.04,90,5.19,0,ZA,Splendid Inn Bayshore
219,ambilobe,66.49,-13.20,49.05,82,3.51,0,MG,Hotel National
273,tigzirt,67.33,36.89,4.12,53,4.81,0,DZ,Hôtel Mizrana
276,ahipara,60.13,-35.17,173.17,65,8.01,0,NZ,Endless Summer Lodge - BBH
310,antsohihy,69.84,-14.88,47.99,69,8.21,0,MG,Hotel Belle Vue
349,yafran,68.27,32.06,12.53,34,9.55,0,LY,فندق يفرن السياحي
351,medea,60.80,36.26,2.75,72,2.24,0,DZ,Gazania B&B


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

  
fig.add_layer(markers)
    
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))